# EDDK Output

This notebook further processes the outputs generated by `eddk data.ipynb` and `eddk opensky.ipynb` and generates aggregates, histogram bin values and other metrics.

## Noise Results

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

output_folder = Path("out/eddk")

### Noise Event Count

In [ ]:
df = pd.read_csv(output_folder / "noise events.csv").merge(
    pd.read_csv(
        output_folder / "flights clean.csv",
        usecols=["flight_id", "operation", "time"],
    ),
    on="flight_id",
)
t = df.groupby(["operation", "receptor_id"])["count"].sum().reset_index()
print(t)

### Noise Event Exposure

In [ ]:
# EDDK, year Leq
df = pd.read_csv(output_folder / "EDDK noise events.csv")
df["energy"] = 10 ** (df["sel"] / 10)
df1 = df.groupby(["operation", "receptor_id"])["energy"].sum().reset_index()
df2 = df.groupby("receptor_id")["energy"].sum().reset_index().assign(operation="All")
df = pd.concat([df1, df2]).reset_index()
df["leq"] = 10 * (np.log10(df["energy"]) - np.log10(365 * 24 * 3600))
print(df)

In [ ]:
# Opensky, year Leq
df = pd.read_csv(output_folder / "noise events.csv").merge(
    pd.read_csv(
        output_folder / "flights clean.csv",
        usecols=["flight_id", "operation", "time"],
    ),
    on="flight_id",
)
df["energy"] = 10 ** (df["sel"] / 10)
df["energy"] *= df["count"]
df1 = df.groupby(["operation", "receptor_id"])["energy"].sum().reset_index()
df2 = df.groupby("receptor_id")["energy"].sum().reset_index().assign(operation="All")
df = pd.concat([df1, df2]).reset_index()
df["leq"] = 10 * (np.log10(df["energy"]) - np.log10(365 * 24 * 3600))
print(df)

### Noise Event Histograms

In [ ]:
path = output_folder / "noise histograms.xlsx"

bins_sel = [0, *np.arange(73, 93.0, 0.5), 200]
bins_lamax = [0, *np.arange(63, 87.0, 0.5), 200]

arrival_stations = ["M01", "M05", "M06", "M07", "M08"]
departure_stations = ["M06", "M08", "M14", "M17", "M18"]

df1 = pd.read_csv(output_folder / "EDDK noise events.csv")  # EDDK
df2 = pd.read_csv(output_folder / "noise events.csv").merge(
    pd.read_csv(
        output_folder / "flights clean.csv",
        usecols=["flight_id", "operation", "time"],
    ),
    on="flight_id",
)  # opensky

w = pd.ExcelWriter(path, mode="w", engine="xlsxwriter")
wb = w.book

for op, ns_list in zip(["Arrival", "Departure"], [arrival_stations, departure_stations]):
    for ns in ns_list:
        name = f"{op} {ns}"

        d1 = df1.loc[(df1["operation"] == op) & (df1["receptor_id"] == ns)]
        d2 = df2.loc[(df2["operation"] == op) & (df2["receptor_id"] == ns)]

        b1_sel = pd.cut(d1["sel"], bins=bins_sel, labels=bins_sel[1:]).value_counts().sort_index()
        b2_sel = pd.cut(d2["sel"], bins=bins_sel, labels=bins_sel[1:]).value_counts().sort_index()

        d_sel = pd.concat([b1_sel, b2_sel], axis="columns").reset_index()
        d_sel.columns = ["SEL", "Airport", "Opensky"]
        d_sel["SEL"] = d_sel["SEL"].astype(float)
        d_sel["SEL"].iat[-1] = d_sel["SEL"].iat[-2] + 0.5

        b1_lamax = pd.cut(d1["lamax"], bins=bins_lamax, labels=bins_lamax[1:]).value_counts().sort_index()
        b2_lamax = pd.cut(d2["lamax"], bins=bins_lamax, labels=bins_lamax[1:]).value_counts().sort_index()

        d_lamax = pd.concat([b1_lamax, b2_lamax], axis="columns").reset_index()
        d_lamax.columns = ["LAMAX", "Airport", "Opensky"]
        d_lamax["LAMAX"] = d_lamax["LAMAX"].astype(float)
        d_lamax["LAMAX"].iat[-1] = d_lamax["LAMAX"].iat[-2] + 0.5

        d_sel.to_excel(w, sheet_name=f"{name} sel", index=False)
        d_lamax.to_excel(w, sheet_name=f"{name} lamax", index=False)

        # Chart
        def format_chart(c, x_name):
            c.set_chartarea({"border": {"none": True}})
            c.set_plotarea({"border": {"none": True}})
            c.set_x_axis(
                {
                    "name": x_name,
                    "name_font": {
                        "size": 14,
                        "bold": False,
                        "color": "#595959",
                    },
                    "num_font": {"size": 11, "color": "#595959"},
                    "line": {"none": True},
                    "major_tick_mark": None,
                    "minor_tick_mark": None,
                    "interval_unit": 4,
                }
            )
            c.set_y_axis(
                {
                    "name": "# Noise Events",
                    "name_font": {
                        "size": 14,
                        "bold": False,
                        "color": "#595959",
                    },
                    "num_font": {"size": 11, "color": "#595959"},
                    "line": {"none": True},
                    "major_tick_mark": None,
                    "minor_tick_mark": None,
                    "major_gridlines": {
                        "visible": True,
                        "line": {"color": "#D9D9D9"},
                    },
                }
            )
            c.set_size({"width": 800, "height": 300})
            c.set_legend(
                {
                    "position": "bottom",
                    "font": {"size": 12, "color": "#595959"},
                }
            )
            return c

        def add_series(c, ws_name, bin_size):
            c.add_series(
                {
                    "categories": [ws_name, 1, 0, bin_size, 0],
                    "values": [ws_name, 1, 1, bin_size, 1],
                    "name": "Validation",
                    "gape": 200,
                    "fill": {"color": "#EA9C03"},
                }
            )
            c.add_series(
                {
                    "categories": [ws_name, 1, 0, bin_size, 0],
                    "values": [ws_name, 1, 2, bin_size, 2],
                    "name": "ADS-B",
                    "gape": 200,
                    "fill": {"color": "#30B7A9"},
                }
            )

        # sel
        ws = wb.get_worksheet_by_name(f"{name} sel")
        c = wb.add_chart({"type": "column"})
        add_series(c, f"{name} sel", 41)
        format_chart(c, "SEL dB(A)")
        ws.insert_chart("H2", c)

        # lamax
        ws = wb.get_worksheet_by_name(f"{name} lamax")
        c = wb.add_chart({"type": "column"})
        add_series(c, f"{name} lamax", 49)
        format_chart(c, "LAMAX dB(A)")
        ws.insert_chart("H2", c)
w.close()

### Emissions Histogram

In [ ]:
operation = "Arrival"

df = pd.read_csv(output_folder / "flights emissions.csv").merge(
    pd.read_csv(output_folder / "flights clean.csv", usecols=["flight_id", "operation"]),
    on="flight_id",
)
df = df.loc[(df["operation"] == operation)].drop(columns="operation")
print(df)